* 설치

In [1]:
%%capture --no-stderr
%pip install python-dotenv langchain langchain-openai

* 실행

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o")

In [4]:
# llm 다이렉트로 호출
result = llm.invoke("2 # 3 값은 몇이야?")
print(result)

content='2 # 3의 값은 문맥에 따라 다를 수 있습니다. 일반적인 수학적 연산으로는 정의되지 않은 연산이므로, 특정한 의미나 정의가 필요합니다. 예를 들어, 프로그래밍 언어나 특정 문제의 규칙에 따라 다른 값을 가질 수 있습니다. 추가적인 정보나 문맥을 제공해주시면 더 정확한 답변을 드릴 수 있습니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 87, 'prompt_tokens': 17, 'total_tokens': 104, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_d28bcae782', 'finish_reason': 'stop', 'logprobs': None} id='run-bfe32d14-9a01-4af0-87e4-6641066553c2-0' usage_metadata={'input_tokens': 17, 'output_tokens': 87, 'total_tokens': 104, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [5]:
from langchain_core.tools import tool
from pprint import pprint

@tool
def operator_sharp(a: int, b: int) -> int:
    """calcurate a # b.

    Args:
        a: first int
        b: second int
    """
    #print(f"call by llm operator # ({a}, {b})")
    return a * b

#result = operator_charp({"a":10, "b":20})
#print(result)

llm_with_tools = llm.bind_tools([operator_sharp])
result = llm_with_tools.invoke("2 # 3 값은 몇이야?")
#print(result)
pprint(result.additional_kwargs)

{'refusal': None,
 'tool_calls': [{'function': {'arguments': '{"a":2,"b":3}',
                              'name': 'operator_sharp'},
                 'id': 'call_t41dWQT6J4bFD04KnMILTdaf',
                 'type': 'function'}]}


In [6]:
# chain으로 호출
from operator import attrgetter
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("user", "{input}")
])

chain = prompt | llm_with_tools | attrgetter("tool_calls") | operator_sharp.map()
result = chain.invoke("2 # 3하면 몇이야?")
print(result)


[ToolMessage(content='6', name='operator_sharp', tool_call_id='call_SQnyypvG75MYwS8KGSoaH5n9')]
